## Opens a basemap in image location and prints list of coordinates selected by user
**Idea is to open images from cluster on top of this map, but not sure how to assign local images URL that works with Jupyter on a remote server (i.e. LocalTileServer can't work in this case) Due to this being hosted on a remote server, we are also currently contrained to Python 3.6, so newer tools like geemap don't work**

## Imports

In [ ]:
import os
import sys
from pathlib import Path
import datetime

import rasterio
from rasterio import plot
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling
import matplotlib.pyplot as plt
import shutil
import tempfile
import json
import random
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
from pyproj import Proj, transform
#import cartopy.crs as ccrs
from shapely.geometry import box
from shapely.geometry import shape
from shapely.geometry import MultiPoint
from shapely.geometry import Point
from shapely.geometry import Polygon
import xarray as xr
import base64
from PIL import Image
import csv

from ipywidgets import Label
from ipyleaflet  import Map, GeoData, basemaps, LayersControl, ImageOverlay
import xarray_leaflet
#from localtileserver import get_leaflet_tile_layer, TileClient  ### Note: This doesn't work on remote server
#import geemap   ##Note geemap doesn't work with Python 3.6; can't use on cluster.

%matplotlib inline

In [ ]:
##This is just for data checking. Not actually used in processing sequence so far.
def GetClosestImage(img_dir, imageType, YYYY, DDD):
    '''
    returns path of image closest to day DDD of year YYYY in directory of images
    Currently looks for images with name YYYYDDD* (DDD is day-of-year(1-365))
    or Sentinel images starting with L1C* (Date format is YYYYMMDD, starting at position 19)
    TODO: expand imageType to Landsat
    '''
    
    imgList = []
    
    if imageType == 'TS':
        for img in os.listdir(img_dir):
            if img.startswith(str(YYYY)):
                imgList.append(int(img[4:7]))
    elif imageType == 'L1C':
        for img in os.listdir(img_dir):
            if img.startswith('L1C') and img[19:23]==str(YYYY):
                MM = int(img[23:25])
                DD = int(img[25:27])
                ymd = datetime.datetime(YYYY, MM, DD)
                doy=int(ymd.strftime('%j'))
                imgList.append(doy)
    
    closestDay = min(imgList, key=lambda x:abs(x-DDD))
    print('closest day is: {}'.format(str(YYYY)+str(closestDay)))
    closestMM = (datetime.datetime(YYYY, 1, 1) + datetime.timedelta(closestDay - 1)).month
    closestDD = (datetime.datetime(YYYY, 1, 1) + datetime.timedelta(closestDay - 1)).day
    
    for fname in os.listdir(img_dir):
        if imageType == 'TS':
            if str(YYYY)+str(closestDay) in fname:
                closestimg = os.path.join(img_dir,fname)
        elif imageType == 'L1C':
            if str(YYYY)+'{:02d}'.format(closestMM)+'{:02d}'.format(closestDD) in fname and fname.startswith('L1C') and 'angles' not in fname:
                closestimg = os.path.join(img_dir,fname)
 
    return closestimg

In [ ]:
### For visualizing index image (Process check 1)
def exploreBand(img, band):

    print('plotting {} band of image {}'.format(img, spec_index))
    fig, axarr = plt.subplots(1, 3, figsize=(15,5))
    
    if img.endswith('.tif'):
        with rasterio.open(img) as src:
            TSamp = src.read()
    elif img.endswith('.nc'):
        with xr.open_dataset(img) as xrimg:
            xrcrs = xrimg.crs
            print(xrcrs)  ##ESRI:102033
            #print(xrimg.variables)
            #print(xrimg.dims)
            #print(xrimg.coords)
    
        ### Check that x and y values correspond to UTM coords (THEY DON'T HERE; they are AlbersEA)
        print("Coord range is: y: {}-{}. x: {}-{}".format(
          xrimg[band]["y"].values.min(), 
          xrimg[band]["y"].values.max(),
          xrimg[band]["x"].values.min(), 
          xrimg[band]["x"].values.max()))
    
        ### Get single band (Opens as Dataset)
        xr_idx = xrimg[band]
    
        ##Note outliers (while nodata = None)
        origHist = xr_idx.plot.hist(color="purple")
        #ax.imshow()
        
        xr_idx_clean = xr_idx.where(xr_idx < 10000)
        maskedHist = xr_idx_clean.plot.hist(color="purple")
        #print('The no data value is:', xrimg[spec_index].rio.nodata)
    
        TSamp = xr_idx_clean

    if img.endswith('.nc'):
        xr_idx.plot.hist(color="purple",  ax=axarr[0])
        axarr[0].set_title("original data")
        xr_idx_clean.plot.hist(color="purple", ax=axarr[1])
        axarr[1].set_title("masked data")
        TSamp.plot(x="x",y="y")
        plot.show(TSamp, ax=axarr[2])
        axarr[2].set_title('{} band'.format(band))
        axarr[2].axis('off')
    
    elif img.endswith('.tif'):
        plot.show(TSamp)
        axarr[2].set_title('{} band'.format(band))
        axarr[2].axis('off')
        
    return axarr

In [ ]:
'''
PARAMETERS: modify in Notebook_settings notebook, then run that notebook and this cell to update here
DO not modify this cell
'''

%store -r basicConfig
print("Basic Parameters: \n PrintDate = {} \n brdf_dir = {} \n gridCell = {} \n index_dir = {} \n out_dir = {}"
      .format(basicConfig['today'], basicConfig['brdf_dir'],basicConfig['gridCell'],basicConfig['index_dir'],basicConfig['out_dir']))

%store -r SinglePlotParams
print("Plotting Parameters: \n plotYr = {} \n plotDay = {} \n Viewband = {} \n imageType = {}"
      .format(SinglePlotParams['plotYr'],SinglePlotParams['plotDay'],SinglePlotParams['Viewband'],SinglePlotParams['imageType']))

%store -r InteractivePlotParams
print( "Shapefile = {} \n If point, file is {} \n If poly, file is {}"
     .format (InteractivePlotParams['shpFile'], InteractivePlotParams['PtFile'],InteractivePlotParams['PolyFile']))

In [ ]:
if SinglePlotParams['imageType'] == 'TS':
    SampImg = GetClosestImage(basicConfig['index_dir'],'TS',SinglePlotParams['plotYr'],SinglePlotParams['plotDay'])
else:
    SampImg = GetClosestImage(basicConfig['brdf_dir'],'L1C',SinglePlotParams['plotYr'],SinglePlotParams['plotDay'])

print(SampImg)

In [ ]:
coordList = []

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'click':
        label.value = str(kwargs.get('coordinates'))
        coords =eval(label.value) 
        coordList.append(coords)
        print(coordList)
        
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)

def gammacorr(band, gamma):
    return np.power(band, 1/gamma)


if SampImg.endswith('.tif'):
    with rasterio.open(SampImg) as src:
        TSamp = src.read()
        print('original image is in {}'.format(src.crs))
        
        dst_crs = 'EPSG:4326'
        kwargs = src.meta.copy()
        transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs.update({'crs': dst_crs,'transform': transform,'width': width,'height': height})
        SampImg_ll = (os.path.join(basicConfig['out_dir'],os.path.basename(SampImg)))
        with rasterio.open(SampImg_ll, 'w+', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
    with rasterio.open(SampImg_ll) as src_ll:
        #TSampll = src_ll.read()
        print('new image is in {}'.format(src_ll.crs))
        
        img_centerT = src_ll.xy(src_ll.height // 2, src_ll.width // 2)
        img_center = [img_centerT[1],img_centerT[0]]
        SW = [src_ll.bounds[1],src_ll.bounds[0]]
        NE = [src_ll.bounds[3],src_ll.bounds[2]]
        img_bounds = [SW, NE]
        print('Image center is at: {}'.format(img_center))
        print('SW and NW corners are at: {}'.format(img_bounds))
    
    #with open(SampImg_ll, "rb") as file:
        #base64_encoded = base64.b64encode(file.read())
    
    ###ipyleaflet cannot render .tif files. Convert to .png 
    SampPng = os.path.splitext(os.path.join(basicConfig['out_dir'],os.path.basename(SampImg)))[0] + ".png"
    im = Image.open(SampImg_ll)
    im.thumbnail(im.size)
    im.save(SampPng, "png", quality=100)
    
    relativePath = os.path.join('../',SampPng)
                   
elif SampImg.endswith('.nc'):
    xrimg = xr.open_dataset(SampImg)
    xrcrs = xrimg.crs
    print(xrcrs)
     
    source_crs = InteractivePlotParams['inputCRS']
    target_crs = 'epsg:4326' # Global lat-lon coordinate system (ipyleaflet only uses lat lon)
    crs_to_latlon = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

    #find the new bounds in lat lon
    lonB, latB = crs_to_latlon.transform([xrimg.x.min(), xrimg.x.max()],[xrimg.y.min(), xrimg.y.max()])
    img_bounds = [(latB[0],lonB[0]),(latB[1],lonB[1])]
    img_center = (latB[0]+((latB[1]-latB[0])/2), lonB[0]+((lonB[1]-lonB[0])/2))
    
    ##reproject to Lat Lon, but TODO: need to reduce XY dimensions first
    #X, Y = np.meshgrid(xrimg.x, xrimg.y)
    #lat1, lon1 = albers_to_latlon.transform(X, Y)
    #lon1 = lon1.reshape(X.shape)
    #lat1 = lat1.reshape(Y.shape)
    #xrimg.coords['lat'] = (xrimg.dims, lat)
    #xrimg.coords['lon'] = (xrimg.dims, lon)
    
    #img_gdf = gpd.GeoDataFrame({"geometry":[img_poly],"crs":"EPSG:4326"})
    #bounds_layer = GeoData(geo_dataframe=img_gdf, name='sampImg')
    
    #relativePath = ##TODO: make this point to the layer to add to map
    
    ###TODO: Add methods to view multilayer composite (take methods from 1b)

else:
    print('only set up to read .tif and .nc files at the moment')

m1 = Map(center=img_center, zoom=12, basemap=basemaps.Esri.WorldImagery)

###TODO: Fix this to show sample image (need a url for a local file that works with Jupyter on a remote server...)
#image = ImageOverlay(url=relativePath, bounds=img_bounds)
#m1.add_layer(image);

### To enable display of coordinates on click:
label = Label()
display(label)
m1.on_interaction(handle_interaction)

### TO Add A shapefile to map (optional):
if InteractivePlotParams['shpFile'] != None:
    if InteractivePlotParams['shpFile'] == 'point':
        if os.path.basename(InteractivePlotParams['PtFile']).endswith('.txt'):
            ptsdf = pd.read_table(InteractivePlotParams['PtFile'], index_col=0, sep=",")
            shps = gpd.GeoDataFrame(ptsdf,geometry=gpd.points_from_xy(ptsdf.XCoord,ptsdf.YCoord),crs=InteractivePlotParams['inputCRS'])
        elif os.path.basename(InteractivePlotParams['PtFile']).endswith('.csv'):
            ptsdf = pd.read_csv(InteractivePlotParams['PtFile'], index_col=0)
            shps = gpd.GeoDataFrame(ptsdf,geometry=gpd.points_from_xy(ptsdf.XCoord,ptsdf.YCoord),crs=InteractivePlotParams['inputCRS'])
        else:
            shps = gpd.read_file(InteractivePlotParams['PtFile'])
    elif InteractivePlotParams['shpFile'] == 'poly':
        shps = gpd.read_file(InteractivePlotParams['PolyFile'])
    shps_ll = shps.to_crs("EPSG:4326")
    viewShp = GeoData(geo_dataframe = shps_ll)
    m1.add_layer(viewShp)

m1

In [ ]:
### Convert list of coordinates back to original CRS and print to file:
##TODO: Allow for other CRS' (this is currently hardcoded for Albers Equal Area)
coordListX = []
coordListY = []
transformer = pyproj.Transformer.from_crs("epsg:4326", "esri:102033")
for pt in transformer.itransform(coordList): 
    print('{:.3f} {:.3f}'.format(pt[0],pt[1]))
    coordListX.append(pt[0])
    coordListY.append(pt[1])
    coords = {'XCoord':coordListX,'YCoord':coordListY}
coorddb = pd.DataFrame(coords)
coorddb = coorddb.astype({'XCoord':'float','YCoord':'float'})
pd.DataFrame(coorddb).to_csv(os.path.join(basicConfig['out_dir'],'SelectedCoords.csv'), sep=',', na_rep='NaN', index=True)
print(coorddb)

## Get Values at Coordinates
###TODO: Make this work with any image type (need to clean methods throughout; not L1C & TS but smooth and raw)

In [ ]:
if isinstance(coorddb, pd.DataFrame):
    ptsdf = coorddb
else:
    PtFile=os.path.join(basicConfig['out_dir'],'SelectedCoords.csv')
    ptsdf = pd.read_csv(PtFile)
### pt file is in SA Albers Equal Area Conic (ESRI:102033)

pts = gpd.GeoDataFrame(ptsdf,geometry=gpd.points_from_xy(ptsdf.XCoord,ptsdf.YCoord),crs='esri:102033')
xy = [pts['geometry'].x, pts['geometry'].y]
coords = list(map(list, zip(*xy)))
    
if SinglePlotParams['imageType'] == 'TS':
    img_name = os.path.basename(SampImg)[:7]
    with rasterio.open(SampImg, 'r') as src:
        ptsval = [sample[0] for sample in src.sample(coords)]

elif SinglePlotParams['imageType'] == 'Sentinel' or SinglePlotParams['imageType'] == 'Landsat' or SinglePlotParams['imageType'] == 'All' :
    if os.path.basename(SampImg).startswith('L1C'):
        YYYY = int(os.path.basename(SampImg)[19:23])
        MM = int(os.path.basename(SampImg)[23:25])
        DD = int(os.path.basename(SampImg)[25:27])
    elif os.path.basename(SampImg).startswith('LC') or os.path.basename(SampImg).startswith('LT') or os.path.basename(SampImg).startswith('LE'):
        YYYY = int(os.path.basename(SampImg)[17:21])
        MM = int(os.path.basename(SampImg)[21:23])
        DD = int(os.path.basename(SampImg)[23:25])
ymd = datetime.datetime(YYYY, MM, DD)
doy = ymd.strftime('%j')
img_name = str(YYYY)+doy
xrimg = xr.open_dataset(SampImg)
xr_val = xrimg[SinglePlotParams['viewBand']].where(xrimg[SinglePlotParams['viewBand']] < 10000)

vals=[]
for index, row in pts.iterrows():
    thispt_val = xr_val.sel(x=pts['geometry'].x[index],y=pts['geometry'].y[index], method='nearest', tolerance=30)
    this_val = thispt_val.values
    vals.append(this_val)
pts[SinglePlotParams['Viewband']] = vals
print(pts)

In [ ]:
if basicConfig['spec_index'] == 'evi2':
    index_val = 2.5 * ((nir_val - b2_val) / (nir_val + 1.0 + 2.4 * b2_val))

## To save an html copy of this notebook with all outputs:

In [ ]:
### Run to print output as html
outName = str(basicConfig['country']+'1c_InteractiveMapSession'
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --output=$outName 1b_ExploreData_OpenImage.ipynb